In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#Data loading and formatting

X = pd.concat([pd.read_pickle(f"feat_splits/feat_split_{i}.pkl") for i in range(1,11)])
y = pd.read_csv("train_labels.csv.zip")
y.set_index("customer_ID", inplace=True)
y = y.reindex(X.index)
y = y.iloc[:,0]

In [3]:
#Creating train and test samples

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1)

In [9]:
#Let's start from the random forest

from sklearn.ensemble import RandomForestClassifier

In [11]:
rf = RandomForestClassifier(
    n_estimators=100,
    min_samples_split=15,
    class_weight={0:20,1:1},
    oob_score=True,
    n_jobs=-1,
    verbose=1,
    warm_start=True
)

In [25]:
rf.n_estimators = 500

In [28]:
rf.n_estimators

500

In [31]:
rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  7.2min finished


RandomForestClassifier(class_weight={0: 20, 1: 1}, min_samples_split=15,
                       n_estimators=500, n_jobs=-1, oob_score=True, verbose=1,
                       warm_start=True)

In [32]:
#quality metrics on test data (from the train sample)

from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score

y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:,rf.classes_[rf.classes_ == 1][0]]
scores = pd.DataFrame(
        {"scores" : [rf.oob_score_, rf.score(X_test, y_test), roc_auc_score(y_test, y_pred_proba), f1_score(y_test, y_pred)]},
        index = ["oob_score", "accuracy", "roc_auc", "f1_score"]
)


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    0.9s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    0.9s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed: 

In [33]:
scores

,scores
oob_score,0.868108
accuracy,0.865071
roc_auc,0.938121
f1_score,0.781651


In [36]:
#Data export
import pickle

with open("RandomForest_model_2.pkl", 'wb') as file: 
     pickle.dump(rf, file)
        
        

In [4]:
from catboost import CatBoostClassifier

In [8]:
cat = CatBoostClassifier(
    iterations=3000,
    learning_rate=0.02,
    depth=3,
)


In [9]:
cat.fit(
    X=X_train,
    y=y_train,
    plot="True",
    early_stopping_rounds=30, 
    eval_set=(X_test, y_test)
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6752108	test: 0.6752832	best: 0.6752832 (0)	total: 74ms	remaining: 3m 41s
1:	learn: 0.6568038	test: 0.6569553	best: 0.6569553 (1)	total: 147ms	remaining: 3m 40s
2:	learn: 0.6416317	test: 0.6418288	best: 0.6418288 (2)	total: 218ms	remaining: 3m 38s
3:	learn: 0.6266524	test: 0.6268718	best: 0.6268718 (3)	total: 291ms	remaining: 3m 38s
4:	learn: 0.6121567	test: 0.6124795	best: 0.6124795 (4)	total: 364ms	remaining: 3m 37s
5:	learn: 0.5987203	test: 0.5990473	best: 0.5990473 (5)	total: 439ms	remaining: 3m 38s
6:	learn: 0.5859466	test: 0.5862591	best: 0.5862591 (6)	total: 507ms	remaining: 3m 36s
7:	learn: 0.5736666	test: 0.5740438	best: 0.5740438 (7)	total: 577ms	remaining: 3m 35s
8:	learn: 0.5620170	test: 0.5624415	best: 0.5624415 (8)	total: 651ms	remaining: 3m 36s
9:	learn: 0.5505956	test: 0.5510369	best: 0.5510369 (9)	total: 723ms	remaining: 3m 36s
10:	learn: 0.5396258	test: 0.5401770	best: 0.5401770 (10)	total: 790ms	remaining: 3m 34s
11:	learn: 0.5308919	test: 0.5314311	best:

94:	learn: 0.3097995	test: 0.3104883	best: 0.3104883 (94)	total: 6.43s	remaining: 3m 16s
95:	learn: 0.3092846	test: 0.3099439	best: 0.3099439 (95)	total: 6.49s	remaining: 3m 16s
96:	learn: 0.3086620	test: 0.3092987	best: 0.3092987 (96)	total: 6.56s	remaining: 3m 16s
97:	learn: 0.3080495	test: 0.3086617	best: 0.3086617 (97)	total: 6.62s	remaining: 3m 15s
98:	learn: 0.3075030	test: 0.3081059	best: 0.3081059 (98)	total: 6.68s	remaining: 3m 15s
99:	learn: 0.3070430	test: 0.3076296	best: 0.3076296 (99)	total: 6.74s	remaining: 3m 15s
100:	learn: 0.3064631	test: 0.3070400	best: 0.3070400 (100)	total: 6.8s	remaining: 3m 15s
101:	learn: 0.3059205	test: 0.3065127	best: 0.3065127 (101)	total: 6.86s	remaining: 3m 15s
102:	learn: 0.3054429	test: 0.3060292	best: 0.3060292 (102)	total: 6.92s	remaining: 3m 14s
103:	learn: 0.3049585	test: 0.3055746	best: 0.3055746 (103)	total: 6.98s	remaining: 3m 14s
104:	learn: 0.3043674	test: 0.3050036	best: 0.3050036 (104)	total: 7.06s	remaining: 3m 14s
105:	learn: 

186:	learn: 0.2812267	test: 0.2816301	best: 0.2816301 (186)	total: 12.1s	remaining: 3m 1s
187:	learn: 0.2810366	test: 0.2814449	best: 0.2814449 (187)	total: 12.1s	remaining: 3m 1s
188:	learn: 0.2808780	test: 0.2812707	best: 0.2812707 (188)	total: 12.2s	remaining: 3m 1s
189:	learn: 0.2807492	test: 0.2811210	best: 0.2811210 (189)	total: 12.3s	remaining: 3m 1s
190:	learn: 0.2805936	test: 0.2809313	best: 0.2809313 (190)	total: 12.3s	remaining: 3m 1s
191:	learn: 0.2804398	test: 0.2807953	best: 0.2807953 (191)	total: 12.4s	remaining: 3m 1s
192:	learn: 0.2802744	test: 0.2806033	best: 0.2806033 (192)	total: 12.4s	remaining: 3m 1s
193:	learn: 0.2801302	test: 0.2804632	best: 0.2804632 (193)	total: 12.5s	remaining: 3m
194:	learn: 0.2799987	test: 0.2803628	best: 0.2803628 (194)	total: 12.6s	remaining: 3m
195:	learn: 0.2798182	test: 0.2801871	best: 0.2801871 (195)	total: 12.6s	remaining: 3m
196:	learn: 0.2796854	test: 0.2800511	best: 0.2800511 (196)	total: 12.7s	remaining: 3m
197:	learn: 0.2795581	

277:	learn: 0.2711199	test: 0.2713773	best: 0.2713773 (277)	total: 17.5s	remaining: 2m 51s
278:	learn: 0.2710306	test: 0.2712885	best: 0.2712885 (278)	total: 17.6s	remaining: 2m 51s
279:	learn: 0.2709260	test: 0.2711870	best: 0.2711870 (279)	total: 17.7s	remaining: 2m 51s
280:	learn: 0.2708380	test: 0.2711094	best: 0.2711094 (280)	total: 17.7s	remaining: 2m 51s
281:	learn: 0.2707787	test: 0.2710412	best: 0.2710412 (281)	total: 17.8s	remaining: 2m 51s
282:	learn: 0.2707122	test: 0.2709754	best: 0.2709754 (282)	total: 17.8s	remaining: 2m 51s
283:	learn: 0.2706488	test: 0.2708957	best: 0.2708957 (283)	total: 17.9s	remaining: 2m 50s
284:	learn: 0.2705884	test: 0.2708434	best: 0.2708434 (284)	total: 17.9s	remaining: 2m 50s
285:	learn: 0.2705148	test: 0.2707711	best: 0.2707711 (285)	total: 18s	remaining: 2m 50s
286:	learn: 0.2704585	test: 0.2707114	best: 0.2707114 (286)	total: 18s	remaining: 2m 50s
287:	learn: 0.2703869	test: 0.2706351	best: 0.2706351 (287)	total: 18.1s	remaining: 2m 50s
288

...

2449:	learn: 0.2395304	test: 0.2484583	best: 0.2484583 (2449)	total: 2m 23s	remaining: 32.3s
2450:	learn: 0.2395244	test: 0.2484586	best: 0.2484583 (2449)	total: 2m 24s	remaining: 32.3s
2451:	learn: 0.2395174	test: 0.2484533	best: 0.2484533 (2451)	total: 2m 24s	remaining: 32.2s
2452:	learn: 0.2395106	test: 0.2484548	best: 0.2484533 (2451)	total: 2m 24s	remaining: 32.1s
2453:	learn: 0.2395036	test: 0.2484639	best: 0.2484533 (2451)	total: 2m 24s	remaining: 32.1s
2454:	learn: 0.2394980	test: 0.2484662	best: 0.2484533 (2451)	total: 2m 24s	remaining: 32s
2455:	learn: 0.2394908	test: 0.2484642	best: 0.2484533 (2451)	total: 2m 24s	remaining: 32s
2456:	learn: 0.2394837	test: 0.2484580	best: 0.2484533 (2451)	total: 2m 24s	remaining: 31.9s
2457:	learn: 0.2394756	test: 0.2484593	best: 0.2484533 (2451)	total: 2m 24s	remaining: 31.9s
2458:	learn: 0.2394689	test: 0.2484566	best: 0.2484533 (2451)	total: 2m 24s	remaining: 31.8s
2459:	learn: 0.2394609	test: 0.2484543	best: 0.2484533 (2451)	total: 2m 24

2539:	learn: 0.2389572	test: 0.2483680	best: 0.2483680 (2539)	total: 2m 29s	remaining: 27s
2540:	learn: 0.2389514	test: 0.2483644	best: 0.2483644 (2540)	total: 2m 29s	remaining: 26.9s
2541:	learn: 0.2389456	test: 0.2483621	best: 0.2483621 (2541)	total: 2m 29s	remaining: 26.9s
2542:	learn: 0.2389353	test: 0.2483606	best: 0.2483606 (2542)	total: 2m 29s	remaining: 26.8s
2543:	learn: 0.2389293	test: 0.2483615	best: 0.2483606 (2542)	total: 2m 29s	remaining: 26.7s
2544:	learn: 0.2389226	test: 0.2483570	best: 0.2483570 (2544)	total: 2m 29s	remaining: 26.7s
2545:	learn: 0.2389161	test: 0.2483575	best: 0.2483570 (2544)	total: 2m 29s	remaining: 26.6s
2546:	learn: 0.2389067	test: 0.2483543	best: 0.2483543 (2546)	total: 2m 29s	remaining: 26.6s
2547:	learn: 0.2388999	test: 0.2483530	best: 0.2483530 (2547)	total: 2m 29s	remaining: 26.5s
2548:	learn: 0.2388940	test: 0.2483544	best: 0.2483530 (2547)	total: 2m 29s	remaining: 26.5s
2549:	learn: 0.2388880	test: 0.2483512	best: 0.2483512 (2549)	total: 2m 

2630:	learn: 0.2383793	test: 0.2482569	best: 0.2482569 (2630)	total: 2m 34s	remaining: 21.6s
2631:	learn: 0.2383725	test: 0.2482493	best: 0.2482493 (2631)	total: 2m 34s	remaining: 21.6s
2632:	learn: 0.2383653	test: 0.2482511	best: 0.2482493 (2631)	total: 2m 34s	remaining: 21.5s
2633:	learn: 0.2383579	test: 0.2482453	best: 0.2482453 (2633)	total: 2m 34s	remaining: 21.5s
2634:	learn: 0.2383512	test: 0.2482386	best: 0.2482386 (2634)	total: 2m 34s	remaining: 21.4s
2635:	learn: 0.2383461	test: 0.2482366	best: 0.2482366 (2635)	total: 2m 34s	remaining: 21.3s
2636:	learn: 0.2383403	test: 0.2482332	best: 0.2482332 (2636)	total: 2m 34s	remaining: 21.3s
2637:	learn: 0.2383351	test: 0.2482371	best: 0.2482332 (2636)	total: 2m 34s	remaining: 21.2s
2638:	learn: 0.2383289	test: 0.2482365	best: 0.2482332 (2636)	total: 2m 34s	remaining: 21.2s
2639:	learn: 0.2383222	test: 0.2482345	best: 0.2482332 (2636)	total: 2m 34s	remaining: 21.1s
2640:	learn: 0.2383168	test: 0.2482344	best: 0.2482332 (2636)	total: 2

2719:	learn: 0.2378269	test: 0.2481513	best: 0.2481513 (2719)	total: 2m 39s	remaining: 16.4s
2720:	learn: 0.2378191	test: 0.2481462	best: 0.2481462 (2720)	total: 2m 39s	remaining: 16.4s
2721:	learn: 0.2378127	test: 0.2481429	best: 0.2481429 (2721)	total: 2m 39s	remaining: 16.3s
2722:	learn: 0.2378063	test: 0.2481477	best: 0.2481429 (2721)	total: 2m 39s	remaining: 16.2s
2723:	learn: 0.2378013	test: 0.2481467	best: 0.2481429 (2721)	total: 2m 39s	remaining: 16.2s
2724:	learn: 0.2377960	test: 0.2481458	best: 0.2481429 (2721)	total: 2m 39s	remaining: 16.1s
2725:	learn: 0.2377898	test: 0.2481474	best: 0.2481429 (2721)	total: 2m 39s	remaining: 16.1s
2726:	learn: 0.2377828	test: 0.2481459	best: 0.2481429 (2721)	total: 2m 39s	remaining: 16s
2727:	learn: 0.2377785	test: 0.2481449	best: 0.2481429 (2721)	total: 2m 39s	remaining: 15.9s
2728:	learn: 0.2377717	test: 0.2481404	best: 0.2481404 (2728)	total: 2m 39s	remaining: 15.9s
2729:	learn: 0.2377653	test: 0.2481398	best: 0.2481398 (2729)	total: 2m 

2810:	learn: 0.2372815	test: 0.2480572	best: 0.2480572 (2810)	total: 2m 44s	remaining: 11.1s
2811:	learn: 0.2372748	test: 0.2480525	best: 0.2480525 (2811)	total: 2m 44s	remaining: 11s
2812:	learn: 0.2372670	test: 0.2480431	best: 0.2480431 (2812)	total: 2m 44s	remaining: 10.9s
2813:	learn: 0.2372588	test: 0.2480391	best: 0.2480391 (2813)	total: 2m 44s	remaining: 10.9s
2814:	learn: 0.2372535	test: 0.2480414	best: 0.2480391 (2813)	total: 2m 44s	remaining: 10.8s
2815:	learn: 0.2372466	test: 0.2480415	best: 0.2480391 (2813)	total: 2m 44s	remaining: 10.8s
2816:	learn: 0.2372389	test: 0.2480457	best: 0.2480391 (2813)	total: 2m 44s	remaining: 10.7s
2817:	learn: 0.2372339	test: 0.2480461	best: 0.2480391 (2813)	total: 2m 44s	remaining: 10.7s
2818:	learn: 0.2372286	test: 0.2480425	best: 0.2480391 (2813)	total: 2m 45s	remaining: 10.6s
2819:	learn: 0.2372234	test: 0.2480447	best: 0.2480391 (2813)	total: 2m 45s	remaining: 10.5s
2820:	learn: 0.2372178	test: 0.2480494	best: 0.2480391 (2813)	total: 2m 

In [10]:
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score

y_pred = cat.predict(X_test)
y_pred_proba = cat.predict_proba(X_test)[:,cat.classes_[cat.classes_ == 1][0]]
scores = pd.DataFrame(
        {"scores" : [ cat.score(X_test, y_test), roc_auc_score(y_test, y_pred_proba), f1_score(y_test, y_pred)]},
        index = [ "accuracy", "roc_auc", "f1_score"]
)
scores

,scores
accuracy,0.889449
roc_auc,0.955970
f1_score,0.824025


In [36]:
#Data export

with open("CatBoostClassifier_model.pkl", 'wb') as file: 
     pickle.dump(cat, file)